In [ ]:
import numpy as np
import pandas as pd


In [3]:
pip install pandas

In [4]:
pip install numpy

In [ ]:
# напишіть функцію гіпотези лінійної регресії у векторному вигляді;
import numpy as np

def hypothesis(X, w):
    """
    Обчислює гіпотезу лінійної регресії.

    Параметри:
    X (numpy.ndarray): Матриця ознак (розмірність m x n), де m - кількість прикладів, n - кількість ознак.
    w (numpy.ndarray): Вектор параметрів моделі (розмірність n).

    Повертає:
    numpy.ndarray: Вектор прогнозів (розмірність m).
    """

    return np.dot(X, w)

In [ ]:
#створіть функцію для обчислення функції втрат у векторному вигляді;
def mean_squared_error(w, X, y):
    m = len(y)
    hyp=hypothesis(w,X)
    error=hyp-y
    return np.sum(error**2) / (2 * m)

In [ ]:
#реалізуйте один крок градієнтного спуску;
def gradient_descent_step(w, X, y, learning_rate):
    """
    Один шаг градиентного спуска для линейной регрессии с использованием MSE.

    Параметры:
    - X (numpy.ndarray): Матрица признаков (размерность m x n).
    - y_true (numpy.ndarray): Вектор истинных значений (размерность m).
    - theta (numpy.ndarray): Вектор параметров модели (размерность n).
    - learning_rate (float): Скорость обучения (размер шага).

    Возвращает:
    - w: Обновленный вектор параметров модели после одного шага градиентного спуска.
    """


    # Считаем прогнозы модели
    m = len(y)
    hyp=hypothesis(w, X)
    error=hyp-y

    # Вычисляем градиент функции
    gradient = np.dot(X.T, error) / m

   # Обновляем параметры модели
    w=w-learning_rate * gradient

    return w


In [5]:
import pandas as pd
url = 'https://drive.google.com/file/d/1e_sSgt8I_Fz50_xFgGpMW07_tZsfa_su/view'
url = "https://drive.google.com/uc?id=" + url.split("/")[-2]
data = pd.read_csv(url)
data

#url.split("/") - это операция разделения строки url на части с использованием символа / в качестве разделителя. Это создает список подстрок, разделенных /.
#[-2] - это индексация, которая выбирает предпоследний элемент из списка, который был создан разделением строки /.

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished
...,...,...,...,...,...,...,...,...,...,...,...,...,...
540,1820000,3000,2,1,1,yes,no,yes,no,no,2,no,unfurnished
541,1767150,2400,3,1,1,no,no,no,no,no,0,no,semi-furnished
542,1750000,3620,2,1,1,yes,no,no,no,no,0,no,unfurnished
543,1750000,2910,3,1,1,no,no,no,no,no,0,no,furnished


In [71]:
#знайдіть найкращі параметри w для датасету прогнозуючу ціну на будинок залежно від площі, кількості ванних кімнат та кількості спалень;
# Варіант 1
from sklearn.linear_model import LinearRegression

#нормализация данных
def normalize(data):
    return (data - data.min()) / (data.max() - data.min())

def denormalize_data(normalized_data, original_min, original_max):
    return normalized * (original_max - original_min) + original_min

# определение признаков модели
sign = ['area', 'bathrooms', 'bedrooms',]
normalized = normalize(data[sign]) #номализованные данные
target = normalize(data['price']) # целевая переменная

model = LinearRegression()
model.fit(normalized, target)

print(f'Intercept: {model.intercept_}')
print(f'Coefficients from scikit-learn: {model.coef_}')


Intercept: 0.04282739976995395
Coefficients from scikit-learn: [0.47714269 0.36001286 0.17611257]


In [81]:
#знайдіть найкращі параметри w для датасету прогнозуючу ціну на будинок залежно від площі, кількості ванних кімнат та кількості спалень;
# Варіант 2
import numpy as np

# Нормализация данных
def normalize(data):
    return (data - data.min()) / (data.max() - data.min())

def denormalize_data(normalized_data, original_min, original_max):
    return normalized_data * (original_max - original_min) + original_min

# Определение признаков модели
sign = ['area', 'bathrooms', 'bedrooms']
normalized = normalize(data[sign])  # Нормализованные данные
target = normalize(data['price'])  # Нормализированная целевая переменная

# Инициализация параметров модели (весов и интерсепта)
num_features = len(sign)
initial_weights = np.zeros(num_features + 1)  # +1 для интерсепта

# Добавление столбца с единицами для интерсепта
X_with_intercept = np.column_stack((np.ones(len(normalized)), normalized))

# Гиперпараметры градиентного спуска
learning_rate = 0.001
num_iterations = 1000

# Обучение модели с помощью градиентного спуска
weights = initial_weights.copy()
for i in range(num_iterations):
    weights = gradient_descent_step(weights, X_with_intercept, target, learning_rate)

#Восстановление исходных (ненормализованных) весов
denormalized_weights = denormalize_data(weights[1:], data[sign].min(), data[sign].max())
intercept = denormalize_data(weights[0], data['price'].min(), data['price'].max())

#print(f'Learned Coefficients: {denormalized_weights}')
#print(f'Intercept: {intercept}')

# Вычисление коэффициентов с использованием нормального уравнения
X_with_intercept = np.column_stack((np.ones(len(normalized)), data[sign]))
coefficients_normal_equation = np.linalg.inv(X_with_intercept.T @ X_with_intercept) @ X_with_intercept.T @ target

print(f'Intercept from normal equation: {coefficients_normal_equation[0]}')
print(f'Coefficients from normal equation: {coefficients_normal_equation[1:]}')


Intercept from normal equation: -0.16650836429719731
Coefficients from normal equation: [3.27933120e-05 1.20004286e-01 3.52225137e-02]


In [ ]:
target = denormalize_data(normalized, data['price'].min(), data['price'].max())
target

,area,bathrooms,bedrooms
0,6.330309e+06,5600000.0,8680000.0
1,7.552784e+06,13300000.0,8680000.0
2,8.346598e+06,5600000.0,6370000.0
3,6.393814e+06,5600000.0,8680000.0
4,6.330309e+06,1750000.0,8680000.0
...,...,...,...
540,2.821649e+06,1750000.0,4060000.0
541,2.345361e+06,1750000.0,6370000.0
542,3.313814e+06,1750000.0,4060000.0
543,2.750206e+06,1750000.0,6370000.0


In [62]:
# знайдіть ці ж параметри за допомогою аналітичного рішення;


# Добавляем столбец единиц к нормализованным данным для Intercept
ones_column = np.ones((normalized.shape[0], 1))
X = np.hstack((ones_column, normalized))

# Вычисляем параметры модели
XTX = X.T.dot(X)  # Произведение X^T на X
XTy = X.T.dot(target)  # Произведение X^T на целевую переменную

# Вычисляем параметры модели с использованием обратной матрицы
w = np.linalg.inv(XTX).dot(XTy)

# Выводим параметры модели
intercept = w[0]
coefficients = w[1:]

print(f'Intercept from normal equation: {intercept}')
print(f'Coefficients from normal equation: {coefficients}')

Intercept from normal equation: 0.04282739976994998
Coefficients from normal equation: [0.47714269 0.36001286 0.17611257]
